In [1]:
import os
import csv
import SimpleITK as sitk
import six
from radiomics import featureextractor, getTestCase
from radiomics.featureextractor import RadiomicsFeatureExtractor
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt 
import pydicom
from pydicom import dcmread
import dicom2nifti
import pydicom_seg
import io
import tempfile
import gc
import time
import shutil
import openpyxl
from dicomweb_client.api import DICOMwebClient
import dicom2nifti.settings as settings

In [2]:
settings. disable_validate_slice_increment()
#pydicom.config.pixel_data_handlers = ['gdcm_handler']

# get the start timen_jobs

st = time.time()
#tmp_dir = os.getenv("SLURM_TMPDIR")                
# Define the float format
float_format = '%.5f'
#main_dir= "/project/166726142/lesee/Synergic-Radiomics/"
main_dir= "/home/ulaval.ca/lesee/projects/Project-CNET/"
params = '/project/166726142/lesee/Synergic-Radiomics/src/Params_test.yaml'
auth_token= 'jsQXPhk1MDJDRCMSmVb4No'
header = {'Authorization': 'Bearer ' + auth_token}

#giving the path of web-based dicom files
url="https://platform.paradim.science/api"
client_dcm = DICOMwebClient(url = url, headers = header)
#data =[]

#find the first 'limit' number of studies
data = pd.read_excel(os.path.join(main_dir,'data/LORD-CNET-rads-clinical-combat.xlsx'))
patient_names = data['PatientName']
#patient_names = ['BT-10042', 'BT-09793', 'BT-08983', 'BT-08888', 'BT-08218', 'BT-07595', 'BT-07020', 'BT-06821', 'BT-06231', 'BT-05885', '4745', 'BT-05233', 'BT-05232', 'BT-05164', 'BT-05100', 'BT-04532', 'BT-04227', 'BT-03904', 'BT-03434', 'BT-02528', 'BT-02396']
#patient_names = ['BT-10042']
all_studies = []
#studies_no_seg = []
#studies_two_seg = []

for patient_name in patient_names:
    search_filters = {'PatientName': patient_name}
    studies = client_dcm.search_for_studies(search_filters=search_filters)
    all_studies.extend(studies)

print(len(all_studies))
#all_studies = client_dcm.search_for_studies(limit = 2)
#all_studies = client_dcm.search_for_studies(limit = 1, get_remaining=False)
#all_studies = client_dcm.search_for_studies(offset=1290)
#series = client_dcm.search_for_series('2.25.117565372918152223031330627690300987977')
#print(len(series))

#instances = client_dcm.search_for_instances(
#    study_instance_uid='2.25.117565372918152223031330627690300987977',
#    series_instance_uid='2.25.306173708352101391281533852372208059598'
#)

#print(len(instances))

91


In [3]:
   
studies_dicom = [pydicom.dataset.Dataset.from_json(d) for d in all_studies]

seg_modality = ['SEG']

#header = ['PatientName', 'patientID' 'StudyInstanceUID'] 
#print(len(studies_dicom))

In [20]:
combat_params_total = []
index_studies = []
combat_key = ['PatientName', 'manufacturer', 'model', 'kernel', 'slice_thickness', 'pixel_spacing']

#to check studies case by case
for num_std , study_dicom in enumerate(studies_dicom): 
    print('Study No. = ', num_std+1 )
    study_instance_uid = study_dicom['StudyInstanceUID'].value
    patient_ID = study_dicom['PatientID'].value
    patient_name = study_dicom['PatientName'].value         
    print('Patient Name = ', patient_name )
    metadata_study = client_dcm.retrieve_study_metadata(study_instance_uid = study_instance_uid)
    modality_values = study_dicom["00080061"]
    if any('SEG' in modality for modality in modality_values):
#        print(modality_values)
        for meta_instance in metadata_study:
        #Find the SEG (Segmentation) object with the SOP Class UID which is typically '1.2.840.10008.5.1.4.1.1.66.4' for SEG files.            
            if meta_instance['00080016']['Value'][0] == '1.2.840.10008.5.1.4.1.1.66.4':
                # This is a SEG file
                SeriesInstanceUID_seg = meta_instance["00081115"]["Value"][0]['0020000E']['Value'][0]           
            else:
                continue    
    
        #find the segmentation dicom dataset        
        series = client_dcm.search_for_series(study_instance_uid=study_instance_uid)
    #    print(series[6])
        num_seg = 0
        for serie in series:
            series_CT_dicom = pydicom.dataset.Dataset.from_json(serie)
            if series_CT_dicom.Modality in seg_modality:
                num_seg +=1
#                print(series_CT_dicom.SeriesDescription)
#*************************************************************************
            # Check for segmentation instances in this series
#                segmentation_instances = client_dcm.retrieve_series(study_instance_uid=study_instance_uid,
#                    series_instance_uid=series_CT_dicom['SeriesInstanceUID'].value)
#                segment_metadata = segmentation_instances[0].SegmentSequence
 #               print(segment_metadata)

#                series_date_str = segmentation_instances[0]["00080021"]
#                series_time_str = segmentation_instances[0]["00080031"]
#                print(series_date_str)
#                print(series_time_str)

#***********************************************************************   

            #find the references CT series corresponding to the SEG binary image 
            if not series_CT_dicom.Modality in seg_modality:
                instances = client_dcm.retrieve_series(study_instance_uid = study_instance_uid,
                    series_instance_uid = series_CT_dicom['SeriesInstanceUID'].value)
                
                if instances[0][0x0020000e].value == SeriesInstanceUID_seg:
                    
                    print(series_CT_dicom.SeriesDescription)
#                    print(instances[0])
                    manufacturer = instances[0][0x00080070].value
                    model_name = instances[0][0x00081090].value
                    convolution_kernel = instances[0][0x00181210].value
                    slice_thickness = instances[0][0x00180050].value
                    pixel_spacing = instances[0][0x00280030].value
                    
                    combat_param = [patient_name, manufacturer, model_name, convolution_kernel, slice_thickness, pixel_spacing]
                    combat_params_total.append(combat_param)
                    index_studies.append(num_std+1)

                    print(f"Manufacturer: {manufacturer}")
                    print(f"Model Name: {model_name}")
                    print(f"Kernel: {convolution_kernel}")
                    print(f"Slice Thickness: {slice_thickness}")
                    print(f"Pixel Spacing: {pixel_spacing}")

                    
                    instances_CT = instances
        if num_seg >1:
#            studies_two_seg.append(patient_name)
            print('Patient Name with more than 1 segmentation file = ', patient_name )
            continue
    else: 
#        studies_no_seg.append(patient_name)

        print ("no SEG file for this study")
        continue

        # Retrieve CT scanner information    
    #    for meta_instance in metadata_study:

    #        manufacturer = meta_instance.get('00080070', {}).get('Value', [''])[0]

            # Now `manufacturer` contains the name of the equipment manufacturer
    #        print(f"Scanner Manufacturer: {manufacturer}")
#print("studies with two SEG files = ", studies_two_seg)
#print("studies with no SEG file = ", studies_no_seg)
 
writer = pd.ExcelWriter(os.path.join(main_dir,'results/combat_params.xlsx'), engine='xlsxwriter')


wb  = writer.book
df = pd.DataFrame(data=combat_params_total, index = index_studies, columns=combat_key)
#df = pd.DataFrame(features_value_total, index = selected_fkeys, columns= features_key)          #put into a dataframe format
df.to_excel(writer, sheet_name="combat params")                                     ## write into excel
wb.close()

Study No. =  1
Patient Name =  BT-10194
Thorax avec ax
Manufacturer: SIEMENS
Model Name: SOMATOM Drive
Kernel: Br43f
Slice Thickness: 2
Pixel Spacing: [0.69921875, 0.69921875]
Study No. =  2
Patient Name =  BT-10178
Thorax avec FL ax
Manufacturer: SIEMENS
Model Name: SOMATOM Drive
Kernel: Br43f
Slice Thickness: 2
Pixel Spacing: [0.65625, 0.65625]


In [ ]:
data_features =[]
index_studies =[]
#with open(os.path.join(main_dir,'results/radiomicsfeatures_kheops_SynergiQc_T_10_test.csv'), mode='w', newline='') as csv_file1:
#    csv_writer_features = csv.writer(csv_file1)
for num_std , study_dicom in enumerate(studies_dicom): 
    try: 
        #find the series instance ID from SEG file in each study
        print('Study No. = ', num_std+1 )
        study_instance_uid = study_dicom['StudyInstanceUID'].value
        patient_ID = study_dicom['PatientID'].value
        patient_name = study_dicom['PatientName'].value         
        print('Patient Name = ', patient_name )
        metadata_study = client_dcm.retrieve_study_metadata(study_instance_uid = study_instance_uid)
        modality_values = study_dicom["00080061"]
        # Check if 'SEG' is in the modality values for this study
        if any('SEG' in modality for modality in modality_values):
        
            for meta_instance in metadata_study:
            #Find the SEG (Segmentation) object with the SOP Class UID which is typically '1.2.840.10008.5.1.4.1.1.66.4' for SEG files.            
                if meta_instance['00080016']['Value'][0] == '1.2.840.10008.5.1.4.1.1.66.4':
                    # This is a SEG file
                    SeriesInstanceUID_seg = meta_instance["00081115"]["Value"][0]['0020000E']['Value'][0]           
                else:
                    continue    
            #find the segmentation dicom dataset        
            series = client_dcm.search_for_series(study_instance_uid=study_instance_uid)
            for serie in series:
                series_CT_dicom = pydicom.dataset.Dataset.from_json(serie)
                if series_CT_dicom.Modality in seg_modality:
                    instances_seg = client_dcm.retrieve_series(study_instance_uid = study_instance_uid,
                        series_instance_uid = series_CT_dicom['SeriesInstanceUID'].value)

                    print(series_CT_dicom.SeriesDescription)
                    #read the SEG dicom dataset as a sitk image file
                    reader_seg = pydicom_seg.MultiClassReader()
                    result = reader_seg.read(instances_seg[0])
                    img_seg = result.image
                    origin = img_seg.GetOrigin()
                    spacing = img_seg.GetSpacing()

                    print("Origin_seg:", origin)
                    print("Spacing_seg:", spacing)

                    segment_metadata = instances_seg[0].SegmentSequence
                    label_name = []
                    label_number =[] 
                    for segment in segment_metadata:
                        segment_number = segment.SegmentNumber
                        segment_label = segment.SegmentLabel
    #                        segment_description = segment.SegmentDescription
                        label_number.append(segment_number)
                        label_name.append(segment_label)
#                    print(label_name)


                #find the references CT series corresponding to the SEG binary image 
                if not series_CT_dicom.Modality in seg_modality:
                    instances = client_dcm.retrieve_series(study_instance_uid = study_instance_uid,
                        series_instance_uid = series_CT_dicom['SeriesInstanceUID'].value)
                    if instances[0][0x0020000e].value == SeriesInstanceUID_seg:
                        print(series_CT_dicom.SeriesDescription)
    #                    print(instances[0])
    #                    manufacturer = instances[0].get('00080070', {}).get('Value', [''])[0]
    #                    print(manufacturer)
                        instances_CT = instances
                        #convert the CT dicom images to one nifti file
                        with tempfile.TemporaryDirectory() as tmp_dir:
    #                        with tempfile.TemporaryDirectory(dir=tmp_dir) as temp_dir:
    #                        with tmp_dir:
        #                   img_CT = dicom2nifti.convert_dicom.dicom_array_to_nifti(instances_CT, output_file = os.path.join(main_dir,str(study_instance_uid)+'_CT.nii'), reorient_nifti=True)["NII"]           
                            img_CT = dicom2nifti.convert_dicom.dicom_array_to_nifti(instances_CT,os.path.join(tmp_dir+'_CT.nii'), reorient_nifti=True)["NII"]                           
                            CT_nifti_path = os.path.join(tmp_dir+'_CT.nii')
        #                   print(CT_nifti_path)
                            #read the CT nifit as a sitk image file                 
                            img_CT_stk = sitk.ReadImage(CT_nifti_path)
                            # Get the origin and spacing
                            origin = img_CT_stk.GetOrigin()
                            spacing = img_CT_stk.GetSpacing()

                            print("Origin_ct:", origin)
                            print("Spacing_ct:", spacing)
                        os.remove(CT_nifti_path)

            # Find number of labels (segments)
            N_segments = len(label_name)

            # Find all labels that start with "Tumor"
            tumor_labels = [label for label in label_name if label.startswith("T")]

            # Loop through all the "Tumor" labels and extract radiomics for each tumor
            for tumor_label in tumor_labels:
            # Extract the name after "Tumor"
                tumor_name = tumor_label[len("Tumor"):]

            # Find the corresponding "Non-Tumor" label with the same name
    #                non_tumor_label = f"Non-tumor{tumor_name}"
    #                if non_tumor_label in label_name:
    #                non_tumor_segment_number = label_number[label_name.index(non_tumor_label)]            
                tumor_segment_number = label_number[label_name.index(tumor_label)]     

                # Extract radiomic features
                extractor = featureextractor.RadiomicsFeatureExtractor(params, additionalInfo=True)
                extractor.settings['n_jobs'] = -1
            #   features = extractor.execute(img_CT_stk, img_seg_resampled, label=int(labels_resampled[-1]))
                features = extractor.execute(img_CT_stk, img_seg, label=int(tumor_segment_number))
                features_key = ['PatientName', 'PatientID', 'StudyInstanceUID' ]
                features_value = [patient_name, patient_ID, study_instance_uid]
                for num , (key, val) in enumerate(six.iteritems(features)):
                    features_key.append(key)
                    features_value.append(val)
                data_features.append(features_value)
                index_studies.append(num_std+1)

            #   print(len(features_key))
            #   print(np.shape(np.asarray(features_value)))
    #            features_value_lists_of_lists.append(features_value)
            #   features_value_total = np.array(features_value_lists_of_lists)
                ind_patientName.append(patient_name )
                ind_patientID.append(patient_ID )
                ind_studyUID.append(study_instance_uid )
    #                print([type(val) for val in features_value])
                # Check if features_value contains a list of floats
    #                if all(isinstance(value, float) for value in features_value):
    #                    print(value)
                # Define the float format
    #                    float_format = '%.5f'
                    # Create a new list of formatted floats
    #                    formatted_features = [float(float_format % value) for value in features_value]
                    # Now, formatted_features contains the float values formatted as specified
    #                else:
                # If features_value contains a list of strings or mixed types, keep it as is
    #                    formatted_features = features_value
                # Write the result as a row in the CSV file
                # Create a new list of formatted floats
    #                features_value_formatted = [float(float_format % value) for value in features_value]
    #                csv_writer_features.writerow([patient_name] + [patient_ID]+[study_instance_uid]+ formatted_features)
    #                print(formatted_features)
        else:
            print ("no SEG file for this study")
            continue
    except Exception as e:
        # Handle any exceptions that occur during the loop
        print(f"An error occurred: {str(e)}")
        #print(features_key)[[patient_name, patient_ID, study_instance_uid] + features_value]
        # Write the result as a row in the CSV file
        # Create a new list of formatted floats
#                features_value_formatted = [float(float_format % value) for value in features_value]
#             worksheet.append(data_row)
#                csv_writer_features.writerow([patient_name] + [patient_ID]+[study_instance_uid]+ formatted_features)
#                print(formatted_features)


Study No. =  1
Patient Name =  2404
Recon 2: HELICAL
Origin_ct: (-165.0, 188.1553497314453, -408.3999938964844)
Spacing_ct: (0.6445310115814209, 0.6445310115814209, 4.8255815505981445)
Segmentation
Origin_seg: (-165.0, -141.199997, -243.399994)
Spacing_seg: (0.644531, 0.644531, 5.0)
An error occurred: Exception thrown in SimpleITK LabelStatisticsImageFilter_Execute: /tmp/SimpleITK/Code/BasicFilters/src/sitkImageFilter.cxx:61:
sitk::ERROR: Input "labelImage" for "LabelStatisticsImageFilter" has size of [ 512, 512, 41 ] which does not match the primary input's size of [ 512, 512, 87 ]!
Study No. =  2
Patient Name =  BT-02629
Thorax  1.0  B40f
Origin_ct: (-153.1923828125, -7.8076171875, 1047.5)
Spacing_ct: (0.615234375, 0.615234375, 1.0)
Segmentation
Origin_seg: (-153.192383, -322.192383, 1164.5)
Spacing_seg: (0.6152344, 0.6152344, 1.0)
An error occurred: Exception thrown in SimpleITK LabelStatisticsImageFilter_Execute: /tmp/SimpleITK/Code/BasicFilters/src/sitkImageFilter.cxx:61:
sitk::ER

In [8]:
print(len(data_features))
print(len(features_value))
print(len(index_studies))
print(len(features_key))

24
1258
24
1258


In [13]:
writer = pd.ExcelWriter(os.path.join(main_dir,'results/radiomicsfeatures_notextrcated_1.xlsx'), engine='xlsxwriter')


wb  = writer.book
df = pd.DataFrame(data=data_features, index = index_studies, columns=features_key)
#df = pd.DataFrame(features_value_total, index = selected_fkeys, columns= features_key)          #put into a dataframe format
df.to_excel(writer, sheet_name="radiomic features")                                     ## write into excel
wb.close()

NameError: name 'features_key' is not defined

In [62]:
# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')


Execution time: 2275.159223794937 seconds
